In [ ]:
import pandas as pd
import os
import sys
root_dir = os.path.dirname(os.path.dirname(os.getcwd())) ## TODO: find way to put this into some global settings
if root_dir not in sys.path:
    sys.path.append(root_dir)

from references import common_cfg, istat_kpi, add_geolocation
import googlemaps


In [ ]:
## Preprocess school list
statali = pd.read_csv('../../data/raw/cultura/stat_anagrScuole1617.csv').set_index('CODICESCUOLA')
paritarie = pd.read_csv('../../data/raw/cultura/parit_anagrScuole1617.csv').set_index('CODICESCUOLA')
assert set(paritarie.columns) < (set(statali.columns)), 'unexpected columns in paritarie'
assert ~any(statali.SEDESCOLASTICA.isnull()), 'missing info for statali'

# add statale/paritaria flag
statali['bStatale'] = True
paritarie['bStatale'] = False

anagrafica = pd.concat([statali, paritarie], axis=0)
# patch tutte le paritarie come sedi scolastiche
anagrafica.SEDESCOLASTICA.fillna('SI', inplace=True)

# drop non-sedi scolastiche
anagrafica = anagrafica[anagrafica.SEDESCOLASTICA=='SI']

In [ ]:
# Filter for each city and append geoloc, then export to interim
listaCitta = common_cfg.city_list

bAddGeoloc = True

for cittaScelta in ['Roma']:
    anagraficaCitta = anagrafica[anagrafica.DESCRIZIONECOMUNE.apply(str.lower) == cittaScelta.lower()]
    anagraficaCitta.to_csv('../../data/interim/%s_anagrafica_scuole.csv' % cittaScelta, sep=';', decimal = ',')
    
    if bAddGeoloc:
        origData = anagraficaCitta.copy()
        # derive target filename
        targetSuffix = '_geoloc'
        targetFile = '../../data/interim/%s_anagrafica_scuole%s.csv' % (citt
                                                                        aScelta, targetSuffix)
        origData.CAPSCUOLA.replace('Non Disponibile', '', inplace=True) # fix missing CAP with empty so that geoloc works
        # build query addresses
        queryAddresses = origData.INDIRIZZOSCUOLA.values + ', ' + origData.CAPSCUOLA.values +  ', %s' % cittaScelta
        APIkey = ''
        # append geoloc and save 
        add_geolocation.append_geolocation_and_save(APIkey, origData, queryAddresses, targetFile)